In [7]:
import os
import sys
#os.environ['CUDA_VISIBLE_DEVICES'] = '0'
notebook_dir = os.path.abspath('')
sys.path.append(os.path.join(notebook_dir, '..'))

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from keras_core import utils
from aepy.data.datasets import load_MNIST
from aepy.data.utils import evaluate
from aepy.models.base.default_architectures import (Decoder_Conv_MNIST, Decoder_MLP,
                                               Encoder_Conv_MNIST, Encoder_MLP)
from aepy.models.vae.vae_model import VAE
from aepy.pipelines.training import TrainingPipeline

In [8]:
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

print(tf.__version__)

No GPU found
2.16.0-dev20231103


In [9]:
# Load MNIST dataset
x_train, y_train, x_test, y_test = load_MNIST(persistant=True)

#x_train = np.expand_dims(x_train, -1).astype("float32") / 255
#x_test = np.expand_dims(x_test, -1).astype("float32") / 255

# Obtaint number of clasess
n_classes = len(set(y_train))

# Convert labels to categorical
y_train = utils.to_categorical(y_train, n_classes)
y_test = utils.to_categorical(y_test, n_classes)

2023-12-20 10:31:06 [INFO]: train-images-idx3-ubyte.gz already exists.
2023-12-20 10:31:06 [INFO]: train-labels-idx1-ubyte.gz already exists.
2023-12-20 10:31:06 [INFO]: t10k-images-idx3-ubyte.gz already exists.
2023-12-20 10:31:06 [INFO]: t10k-labels-idx1-ubyte.gz already exists.


In [10]:
train_data = dict(data=x_train.astype(float), labels=y_train)
test_data = dict(data=x_test.astype(float), labels=y_test)
print(y_test.shape)
print(x_test.shape)

KeyboardInterrupt: 

In [ ]:
# Model creation
model = VAE(input_dim=(x_train.shape[1], x_train.shape[2]), latent_dim=10, downstream_task='classification',
            encoder=Encoder_Conv_MNIST, decoder=Decoder_Conv_MNIST, layers_conf=[32, 64], n_classes=n_classes,
            weight_vae=0.25, weight_clf=20)

2023-12-20 10:22:09 [INFO]: Classificator available for the latent space of the autoencoder


In [ ]:
pipe = TrainingPipeline(name='training_pipeline',
                        model=model, num_epochs=6, batch_size=128)

trained_model = pipe(x=x_train.astype(float), y=y_train)

2023-12-20 10:22:09 [INFO]: +++ training_pipeline +++
2023-12-20 10:22:09 [INFO]: Creating folder in ../output_dir/training_pipeline_2023-12-20_10-22-09


Epoch 1/6


2023-12-20 10:22:10.360514: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.



Epoch 1: loss improved from inf to 29386.04492, saving model to ../output_dir/training_pipeline_2023-12-20_10-22-09/model.weights.h5
469/469 - 137s - 292ms/step - clf_loss: 2.5943 - kl_loss: 110119.3906 - loss: 29386.0449 - reconstruction_loss: 7217.3179
Epoch 2/6


2023-12-20 10:24:27.443720: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.



Epoch 2: loss improved from 29386.04492 to 1835.87000, saving model to ../output_dir/training_pipeline_2023-12-20_10-22-09/model.weights.h5
469/469 - 132s - 281ms/step - clf_loss: 1.0566 - kl_loss: 41.7166 - loss: 1835.8700 - reconstruction_loss: 7217.2324
Epoch 3/6


2023-12-20 10:26:38.888980: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.



Epoch 3: loss improved from 1835.87000 to 1826.79236, saving model to ../output_dir/training_pipeline_2023-12-20_10-22-09/model.weights.h5
469/469 - 131s - 278ms/step - clf_loss: 0.6774 - kl_loss: 35.9561 - loss: 1826.7924 - reconstruction_loss: 7217.0190
Epoch 4/6


2023-12-20 10:28:49.434239: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.


KeyboardInterrupt: 

In [ ]:
model.load_weights(filepath='../output_dir/training_pipeline_2023-11-25_00-50-30/model.weights.h5')

In [ ]:
def plot_label_clusters(vae,  must be equal, but ar
    # Display a 2D plot of the digit classes in the latent space
    _, _, z = vae.predict(data)
    plt.figure(figsize=(12, 10))
    plt.scatter(z[:, 0], z[:, 1], c=labels)
    plt.colorbar()
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.show()
    if include_flow:
      z = vae.flow.inverse(z)
      plt.figure(figsize=(12, 10))
      plt.scatter(z[:, 0], z[:, 1], c=labels)
      plt.colorbar()
      plt.xlabel("z[0]")
      plt.ylabel("z[1]")
      plt.show()

In [ ]:
plot_label_clusters(model, train_data, y_train)
plot_label_clusters(model, test_data, y_test)

In [ ]:
from sklearn.metrics import classification_report

#y_train_predict = model.predict(train_data)
y_test_predict =  model.predict(test_data)

print(np.argmax(y_test_predict['clf'], axis=1))
print(np.argmax(y_test, axis=1))

target_names = ['number 0', 'number 1', 'number 2', 'number 3', 'number 4', 'number 5', 'number 6', 'number 7', 'number 8', 'number 9']

#classification_report(y_train, np.argmax(y_train_predict['clf'], axis=1), target_names=target_names)
print(classification_report(np.argmax(y_test, axis=1), np.argmax(y_test_predict['clf'], axis=1), target_names=target_names))

In [ ]:
y_hat = trained_model.predict(test_data)

evaluate(y_true=np.argmax(test_data['labels'], axis=1), 
         y_hat=np.argmax(y_hat['clf'], axis=1),
         sel_metric=accuracy_score)